In [3]:
import pandas as pd
from pulp import *

In [4]:
data = pd.read_excel("transportation data.xlsx",engine='openpyxl')
data

In [6]:
nov,dec,jan,feb,mar,cost=[],[],[],[],[],[]
rows, cols=11,9
for i in range(rows):
    col = []
    for j in range(cols):
        col.append(0)
    nov.append(col)
    dec.append(col)
    jan.append(col)
    feb.append(col)
    mar.append(col)
    cost.append(col)


In [7]:
data.iloc[11][9]

10.0

In [14]:
for i in range(0,11):
    for j in range(1,10):
        nov[i][j-1]=data.iloc[i][j]
for i in range(12,23):
    for j in range(1,10):
        dec[i-12][j-1]=data.iloc[i][j]       
for i in range(23,34):
    for j in range(1,10):
        jan[i-23][j-1]=data.iloc[i][j]
for i in range(34,45):
    for j in range(1,10):
        feb[i-34][j-1]=data.iloc[i][j]
for i in range(45,56):
    for j in range(1,10):
        mar[i-45][j-1]=data.iloc[i][j]
for i in range(56,67):
    for j in range(1,10):
        cost[i-56][j-1]=data.iloc[i][j]

Nov
50	50	50	150	40
Dec
80	50	50	150	50
Jan
110	50	50	150	50
Feb
110	50	50	150	40
Mar
150	60	100	150	40
Different supplies when changing months(for convenience).


In [16]:
nov

defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
            {'O1': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {}),
             'O2': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {}),
             'O3': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {}),
             'O4': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {}),
             'O6': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {}),
             'O8': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {}),
             'O10': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {}),
             'O11': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
      

In [13]:
#Define Destination and Origin Matrices
Dest= ["D1","D2","D3","D4","D5","D6","D7","D8","D9"]
Origin=["O1","O2","O3","O4","O6","O8","O10","O11","O12","O13","O14"]
demand = {"D1":50,"D2":40,"D3":60,"D4":100,"D5":30,"D6":20,"D7":50,"D8":30,"D9":40}

#Define Demand(up for problem d,e,f down for a,b,c) and Supply(cost and paste different supply prices in different months) Matrices
"""demand = {"D1":60,
          "D2":18,
          "D3":18,
          "D4":19,
          "D5":10,
          "D6":2,
          "D7":20,
          "D8":17,
          "D9":10}"""

supply={"O1":0,"O2":0,"O3":0,"O4":0,"O6":0,"O8":0,"O10":50,"O11":50,"O12":50,"O13":150,"O14":40}#this set is nov supply currently.

#Create dictionaries for different months
cost = makeDict([Origin, Dest], cost, 0)
print(cost)
nov = makeDict([Origin, Dest], nov, 0)
dec = makeDict([Origin, Dest], dec, 0)
jan = makeDict([Origin, Dest], jan, 0)
feb = makeDict([Origin, Dest],feb, 0)
mar = makeDict([Origin, Dest],mar, 0)

#prob1 for maximize sum of route capacity when max capacity of warehouse is considered
prob1=LpProblem("Maximum_Route_capacity_sum_in_constant_amount_of_money", LpMaximize)
# Creates a list of tuples containing all the possible routes for transport
Routes = [(w, b) for w in Origin for b in Dest]
# A dictionary called 'Vars' is created to contain the referenced variables(the routes)
vars = LpVariable.dicts("Route", (Origin,Dest), 0, None, LpInteger)
prob1 += (lpSum([vars[w][b] for (w, b) in Routes]),
    "Sum_of_Routing_Capacity",
)
vars
LpStatus = [prob1.status]

defaultdict(<function __makeDict.<locals>.<lambda> at 0x000001EA88804700>, {'O1': defaultdict(<function __makeDict.<locals>.<lambda> at 0x000001EA88B4C040>, {}), 'O2': defaultdict(<function __makeDict.<locals>.<lambda> at 0x000001EA88B4C670>, {}), 'O3': defaultdict(<function __makeDict.<locals>.<lambda> at 0x000001EA88B4D090>, {}), 'O4': defaultdict(<function __makeDict.<locals>.<lambda> at 0x000001EA888072E0>, {}), 'O6': defaultdict(<function __makeDict.<locals>.<lambda> at 0x000001EA88807C70>, {}), 'O8': defaultdict(<function __makeDict.<locals>.<lambda> at 0x000001EA888070A0>, {}), 'O10': defaultdict(<function __makeDict.<locals>.<lambda> at 0x000001EA88807A30>, {}), 'O11': defaultdict(<function __makeDict.<locals>.<lambda> at 0x000001EA88804B80>, {}), 'O12': defaultdict(<function __makeDict.<locals>.<lambda> at 0x000001EA888069E0>, {}), 'O13': defaultdict(<function __makeDict.<locals>.<lambda> at 0x000001EA88804820>, {}), 'O14': defaultdict(<function __makeDict.<locals>.<lambda> at

for w in Origin:
    prob1 += (
        lpSum([vars[w][b] for b in Dest]) <= supply[w],
    "Sum_of_Products_out_of_warehouses_%s" %w,
    )

# The demand minimum constraints are added to prob for each demand node (project)
for b in Dest:
    prob1 += (
        lpSum([vars[w][b] for w in Origin]) >= demand[b],
        "Sum_of_Products_into_projects%s" % b,
    )
for w in Origin:
    for b in Dest:
        prob1+=(vars[w][b]>=0)

In [12]:
prob1.solve()

# Print the variables optimized value
for v in prob1.variables():
    print(v.name, "=", v.varValue)
    
# The optimised objective function value is printed to the screen
print("Value of Objective Function = ", value(prob1.objective))

PulpSolverError: PuLP: Error while executing glpsol.exe

In [ ]:
#prob for minimize amount of money used in routes(d,e,f)
prob = LpProblem("Minimum_Cost", LpMinimize)

prob += (
    lpSum([vars[w][b] * cost[w][b] for (w, b) in Routes]),
    "Sum_of_Transporting_Costs",
)

In [ ]:
#Prob constraints settings 
# The supply maximum constraints are added to prob for each supply node (warehouses)
for w in Origin:
    prob += (
        lpSum([vars[w][b] for b in Dest]) <= supply[w],
    "Sum_of_Products_out_of_warehouses_%s" %w,
    )


# The demand minimum constraints are added to prob for each demand node (project)
for b in Dest:
    prob += (
        lpSum([vars[w][b] for w in Origin]) >= demand[b],
        "Sum_of_Products_into_projects%s" % b,
    )
"""
 
for b in Origin:
    for w in Dest:
        prob+=(vars[w][b]<=mar[w][b])      

 
for b in Origin:
    for w in Dest:
        prob+=(vars[w][b]<=dec[w][b])

 
for b in Origin:
    for w in Dest:
        prob+=(vars[w][b]<=jan[w][b])        
for b in Origin:
    for w in Dest:
        prob+=(vars[w][b]<=feb[w][b])  
"""
for w in Origin:
    for b in Dest:
        prob+=(vars[w][b]<=nov[w][b])   
for w in Origin:
    for b in Dest:
        prob+=(vars[w][b]>=0)      


In [ ]:
prob.solve()

# Print the variables optimized value
for v in prob.variables():
    print(v.name, "=", v.varValue)
    
# The optimised objective function value is printed to the screen
print("Value of Objective Function = ", value(prob.objective))
    